## Data Gathering
We are going to be using data from only the last season to predict the next season, we could be using data from earlier seasons but that wouldn't give us a good estimate of the teams' quality as teams have changed a lot of the past few years, stats from the last season would be the best estimate to calculate how strong the teams are currently.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import statsmodels.api as sm

loc = 'Dataset//'

data1 = pd.read_csv(loc + 'E0 (1).csv')
# data2 = pd.read_csv('E0 (2).csv')
# data3 = pd.read_csv('E0 (3).csv')
# data4 = pd.read_csv('E0 (4).csv')
# data5 = pd.read_csv('E0 (5).csv')
# data6 = pd.read_csv('E0 (6).csv')
# data7 = pd.read_csv('E0 (7).csv')
# data8 = pd.read_csv('E0 (8).csv')
# data9 = pd.read_csv('E0 (9).csv')
# data10 = pd.read_csv('E0 (10).csv')
# data11 = pd.read_csv('E0 (11).csv')

# data5 = data5.drop(380) # This dataframe has a extra row blank row

dataframes = [data1]

for ind, data in enumerate(dataframes):
    data['Season'] = "{}/{}".format(18 - ind, 19 - ind)

col_list = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS',
            'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'Season']

all_data = pd.concat(dataframes, sort='False')
all_data = all_data[col_list]

def result(HG, AG):
    if HG > AG:
        return -1
    elif HG == AG:
        return 0
    elif HG < AG:
        return 1

all_data['Result'] = all_data.apply(lambda row: result(row.FTHG, row.FTAG), axis = 1) 
print(all_data.shape)
all_data

(380, 18)


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,Season,Result
0,Man United,Leicester,2,1,H,1,0,H,8,13,6,4,11,8,2,5,18/19,-1
1,Bournemouth,Cardiff,2,0,H,1,0,H,12,10,4,1,11,9,7,4,18/19,-1
2,Fulham,Crystal Palace,0,2,A,0,1,A,15,10,6,9,9,11,5,5,18/19,1
3,Huddersfield,Chelsea,0,3,A,0,2,A,6,13,1,4,9,8,2,5,18/19,1
4,Newcastle,Tottenham,1,2,A,1,2,A,15,15,2,5,11,12,3,5,18/19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Liverpool,Wolves,2,0,H,1,0,H,13,7,5,2,3,11,4,1,18/19,-1
376,Man United,Cardiff,0,2,A,0,1,A,26,13,10,4,9,6,11,2,18/19,1
377,Southampton,Huddersfield,1,1,D,1,0,H,10,10,3,3,8,6,4,3,18/19,0
378,Tottenham,Everton,2,2,D,1,0,H,11,17,3,9,10,13,7,4,18/19,0


## Data Wrangling
To get a rouh estimate of how good the teams' attack and defense are, we get the no of goals thye've scored and conceded, home and away. We then calculate a rough estimate of their attack and defense strength, home and away. We then sacle these data from 0 to 1 to help us when calculating their chances for a fixture.

(We did include the no of shots taken in out previous model but found out that it didn't really give us a hige improvement so decided against using it)

In [2]:
home_data = all_data.groupby('HomeTeam').sum()
away_data = all_data.groupby('AwayTeam').sum()
home_data['HAS'] = home_data['FTHG'] / 19 # Home Average Scores
home_data['HAC'] = home_data['FTAG'] / 19 # Home Average Conceded
away_data['AAS'] = away_data['FTAG'] / 19 # Away average Scored
away_data['AAC'] = away_data['FTHG'] / 19 # Away Average Conceded
all_data_backup = all_data
all_data
team_data = pd.DataFrame()
team_data['Team'] = home_data.index

team_data = team_data.merge(home_data['HAS'], left_on='Team', right_index=True)
team_data = team_data.merge(home_data['HAC'], left_on='Team', right_index=True)
# team_data = team_data.merge(home_data['HS'], left_on='Team', right_index=True)
# team_data = team_data.merge(home_data['AS'], left_on='Team', right_index=True)

# team_data['AHSH'] = team_data['HS'] / 19 # Average Home Shots Hit
# team_data['AAST'] = team_data['AS'] / 19 # Average Away Shots Taken
# team_data = team_data.drop(['HS', 'AS'], axis=1)

team_data = team_data.merge(away_data['AAS'], left_on='Team', right_index=True)
team_data = team_data.merge(away_data['AAC'], left_on='Team', right_index=True)
# team_data = team_data.merge(away_data['AS'], left_on='Team', right_index=True)
# team_data = team_data.merge(away_data['HS'], left_on='Team', right_index=True)

# team_data['AASH'] = team_data['AS'] / 19 # Average Away Shots Hit
# team_data['AHST'] = team_data['HS'] / 19 # Average Home Shots Taken
# team_data = team_data.drop(['AS', 'HS'], axis=1)


team_data.set_index("Team", inplace=True)
last_season_table= pd.DataFrame(index=team_data.index)
scaler = MinMaxScaler()
team_data = pd.DataFrame(scaler.fit_transform(team_data), columns=team_data.columns, index=team_data.index)
team_data

,HAS,HAC,AAS,AAC
Team,,,,
Arsenal,0.680851,0.214286,0.730769,0.705882
Bournemouth,0.425532,0.535714,0.538462,1.000000
Brighton,0.191489,0.642857,0.153846,0.617647
Burnley,0.297872,0.785714,0.346154,0.735294
Cardiff,0.234043,1.000000,0.038462,0.588235
Chelsea,0.617021,0.071429,0.461538,0.470588
Crystal Palace,0.191489,0.464286,0.769231,0.558824
Everton,0.425532,0.392857,0.461538,0.411765
Fulham,0.255319,0.928571,0.000000,1.000000


In [3]:
col_list2 = ['HomeTeam', 'AwayTeam']
data_prep = all_data[col_list2]

data_prep = data_prep.merge(team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
data_prep = data_prep.merge(team_data['HAC'], left_on='HomeTeam', right_index=True, how='left') 
data_prep = data_prep.merge(team_data['AAS'], left_on='AwayTeam', right_index=True, how='left') 
data_prep = data_prep.merge(team_data['AAC'], left_on='AwayTeam', right_index=True, how='left') 

# data_prep = data_prep.merge(team_data['AHSH'], left_on='HomeTeam', right_index=True, how='left') 
# data_prep = data_prep.merge(team_data['AHST'], left_on='HomeTeam', right_index=True, how='left') 

# data_prep = data_prep.merge(team_data['AASH'], left_on='AwayTeam', right_index=True, how='left') 
# data_prep = data_prep.merge(team_data['AAST'], left_on='AwayTeam', right_index=True, how='left') 

# data_prep['HAtS'] = data_prep['HAS'] * data_prep['AAC'] * data_prep['AHSH'] * data_prep['AAST']; # Home Attack Strength
# data_prep['AAtS'] = data_prep['HAC'] * data_prep['AAS'] * data_prep['AHST'] * data_prep['AASH']; # Away Attack Strength

data_prep['HAtS'] = (data_prep['HAS'] * data_prep['AAC']); # Home Attack Strength
data_prep['AAtS'] = (data_prep['HAC'] * data_prep['AAS']); # Away Attack Strength

print(data_prep.shape)

data_prep = data_prep.merge(all_data['Result'], left_index=True, right_index=True)

fixture_list = data_prep
data_prep

(380, 8)


,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,Man United,Leicester,0.489362,0.535714,0.576923,0.500000,0.244681,0.309066,-1
1,Bournemouth,Cardiff,0.425532,0.535714,0.038462,0.588235,0.250313,0.020604,-1
2,Fulham,Crystal Palace,0.255319,0.928571,0.769231,0.558824,0.142678,0.714286,1
3,Huddersfield,Chelsea,0.000000,0.750000,0.461538,0.470588,0.000000,0.346154,1
4,Newcastle,Tottenham,0.297872,0.535714,0.807692,0.352941,0.105131,0.432692,1
...,...,...,...,...,...,...,...,...,...
375,Liverpool,Wolves,0.957447,0.000000,0.269231,0.411765,0.394243,0.000000,-1
376,Man United,Cardiff,0.489362,0.535714,0.038462,0.588235,0.287860,0.020604,1
377,Southampton,Huddersfield,0.361702,0.714286,0.000000,1.000000,0.361702,0.000000,0
378,Tottenham,Everton,0.510638,0.214286,0.461538,0.411765,0.210263,0.098901,0


In [4]:
rm_col_list = ['HomeTeam','AwayTeam']
col_list = data_prep.columns.values.tolist()
col_list = [x for x in col_list if x not in rm_col_list]

to_use = data_prep[col_list]
X = [x for x in col_list if x != 'Result']
Y = [x for x in col_list if x == 'Result']

to_use

,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,0.489362,0.535714,0.576923,0.500000,0.244681,0.309066,-1
1,0.425532,0.535714,0.038462,0.588235,0.250313,0.020604,-1
2,0.255319,0.928571,0.769231,0.558824,0.142678,0.714286,1
3,0.000000,0.750000,0.461538,0.470588,0.000000,0.346154,1
4,0.297872,0.535714,0.807692,0.352941,0.105131,0.432692,1
...,...,...,...,...,...,...,...
375,0.957447,0.000000,0.269231,0.411765,0.394243,0.000000,-1
376,0.489362,0.535714,0.038462,0.588235,0.287860,0.020604,1
377,0.361702,0.714286,0.000000,1.000000,0.361702,0.000000,0
378,0.510638,0.214286,0.461538,0.411765,0.210263,0.098901,0


In [5]:
X_data = to_use.drop('Result', axis=1)
Y_data = to_use['Result']

model = XGBClassifier(max_iter=10000)
model.fit(X_data, Y_data)
model.score(X_data, Y_data)

0.7868421052631579

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

model1 = XGBClassifier(max_iter=10000)
model1.fit(X_train, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3, max_iter=10000,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [7]:
predicted = model1.predict(X_test)
predicted

array([-1,  0,  1, -1, -1,  1, -1, -1,  1, -1,  1, -1,  1,  1, -1, -1,  0,
       -1,  1,  0, -1, -1,  1, -1, -1, -1,  1, -1, -1,  0, -1, -1,  1, -1,
       -1, -1,  1, -1, -1, -1,  1,  1,  1, -1,  0,  1, -1, -1, -1, -1, -1,
        1,  0, -1, -1, -1,  1,  0, -1, -1,  1, -1, -1,  1, -1,  0,  1,  1,
       -1, -1, -1, -1,  1,  1, -1,  1], dtype=int64)

In [8]:
acc_score = accuracy_score(Y_test, predicted)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(model1, X_train, Y_train, cv=10)
print("Cross val scores")
print(scores)

Accuracy score: 0.631578947368421
Cross val scores
[0.5483871  0.58064516 0.48387097 0.48387097 0.6        0.46666667
 0.6        0.6        0.6        0.53333333]


In [9]:
score_train = model1.score(X_train, Y_train)
score_test = model1.score(X_test, Y_test)

print("Training set accuracy: ", '%.3f'%(score_train))
print("Test set accuracy: ", '%.3f'%(score_test))

Training set accuracy:  0.819
Test set accuracy:  0.632


In [10]:
predicted = model1.predict(X_data)
predicted
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,-1
1,-1
2,1
3,1
4,1


In [11]:
to_use = to_use.merge(predictedDF, left_index=True, right_index=True)
fixture_list = fixture_list.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result,Predicted
0,Man United,Leicester,0.489362,0.535714,0.576923,0.500000,0.244681,0.309066,-1,-1
1,Bournemouth,Cardiff,0.425532,0.535714,0.038462,0.588235,0.250313,0.020604,-1,-1
2,Fulham,Crystal Palace,0.255319,0.928571,0.769231,0.558824,0.142678,0.714286,1,1
3,Huddersfield,Chelsea,0.000000,0.750000,0.461538,0.470588,0.000000,0.346154,1,1
4,Newcastle,Tottenham,0.297872,0.535714,0.807692,0.352941,0.105131,0.432692,1,1
...,...,...,...,...,...,...,...,...,...,...
375,Liverpool,Wolves,0.957447,0.000000,0.269231,0.411765,0.394243,0.000000,-1,-1
376,Man United,Cardiff,0.489362,0.535714,0.038462,0.588235,0.287860,0.020604,1,-1
377,Southampton,Huddersfield,0.361702,0.714286,0.000000,1.000000,0.361702,0.000000,0,-1
378,Tottenham,Everton,0.510638,0.214286,0.461538,0.411765,0.210263,0.098901,0,-1


In [12]:
last_season_table['Actual Points'] = 0 
last_season_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == -1:
        last_season_table.loc[ht, which] += 3
    elif x == 0:
        last_season_table.loc[ht, which] += 1
        last_season_table.loc[at, which] += 1
    elif x == 1:
        last_season_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Result, x.HomeTeam, x.AwayTeam, 'Actual Points'), axis=1)
fixture_list.apply(lambda x: make_actual_table(x.Predicted, x.HomeTeam, x.AwayTeam, 'Predicted Points'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [13]:
# 18/19 Season actual table
actual_index = last_season_table.sort_values(by='Actual Points', ascending=0).index
predicted_index = last_season_table.sort_values(by='Predicted Points', ascending=0).index

last_season_table.sort_values(by='Actual Points', ascending=0)

,Actual Points,Predicted Points
Team,,
Man City,98,108
Liverpool,97,99
Chelsea,72,81
Tottenham,71,91
Arsenal,70,78
Man United,66,71
Wolves,57,52
Everton,54,74
West Ham,52,64


## Actual vs Predicted Table for the 18/19 Season
As we can see, we did get the champions and the relegation teams right, but missed on a few others here and there.

In [14]:
def acc(act, prd):
    count = 0
    for i, val in enumerate(act):
        if str(act[i]) == str(prd[i]):
            count += 1
    print("Accuracy of Predicted Table to Accurate Table " + str((count/20)*100) + "%")

acc(actual_index,predicted_index)

# 18/19 Season predicted table
last_season_table.sort_values(by='Predicted Points', ascending =0)

Accuracy of Predicted Table to Accurate Table 30.0%


,Actual Points,Predicted Points
Team,,
Man City,98,108
Liverpool,97,99
Tottenham,71,91
Chelsea,72,81
Arsenal,70,78
Everton,54,74
Man United,66,71
West Ham,52,64
Watford,50,52


In [15]:
# Let's bring in the 19/20 season fixture list and try predicting the current season

curr_season = pd.read_csv(loc + "E0.csv")
curr_season['Result'] = curr_season.apply(lambda row: result(row.FTHG, row.FTAG), axis = 1) 
curr_season.append(all_data, sort='False')
curr_season

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Result
0,E0,09/08/2019,20:00,Liverpool,Norwich,4,1,H,4,0,...,-2.25,1.91,1.99,1.94,1.98,1.99,2.07,1.90,1.99,-1
1,E0,10/08/2019,12:30,West Ham,Man City,0,5,A,0,1,...,1.75,1.95,1.95,1.96,1.97,2.07,1.98,1.97,1.92,1
2,E0,10/08/2019,15:00,Bournemouth,Sheffield United,1,1,D,0,0,...,-0.50,1.95,1.95,1.98,1.95,2.00,1.96,1.96,1.92,0
3,E0,10/08/2019,15:00,Burnley,Southampton,3,0,H,0,0,...,0.00,1.87,2.03,1.89,2.03,1.90,2.07,1.86,2.02,-1
4,E0,10/08/2019,15:00,Crystal Palace,Everton,0,0,D,0,0,...,0.25,1.82,2.08,1.97,1.96,2.03,2.08,1.96,1.93,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,E0,04/12/2019,19:30,Leicester,Watford,2,0,H,0,0,...,-1.50,2.03,1.90,2.05,1.88,2.08,1.95,2.00,1.88,-1
144,E0,04/12/2019,19:30,Man United,Tottenham,2,1,H,1,1,...,0.00,1.88,2.05,1.88,2.05,1.94,2.08,1.87,2.02,-1
145,E0,04/12/2019,19:30,Southampton,Norwich,2,1,H,2,0,...,-0.75,2.00,1.93,2.00,1.93,2.05,1.94,2.00,1.89,-1
146,E0,04/12/2019,19:30,Wolves,West Ham,2,0,H,1,0,...,-0.75,1.88,2.05,1.88,2.05,1.95,2.06,1.87,2.01,-1


## Data Wrangling for the new season
Since some of the teams in the current season were promoted form the Championship (Division 2), we dont have their stats in the Premier League, so we will get the stats of those teams by looking at the matches played till Matchday 15 [12/6/2019]. For the teams already who have stayed in the Premier League, we will be using the stats from the last season.

In [16]:
curr_home_data = curr_season.groupby('HomeTeam').sum()
curr_away_data = curr_season.groupby('AwayTeam').sum()
curr_home_data['HAS'] = curr_home_data['FTHG'] / 19 # Home Average Scores
curr_home_data['HAC'] = curr_home_data['FTAG'] / 19 # Home Average Conceded
curr_away_data['AAS'] = curr_away_data['FTAG'] / 19 # Away average Scored
curr_away_data['AAC'] = curr_away_data['FTHG'] / 19 # Away Average Conceded
curr_season_backup = curr_season
curr_team_data = pd.DataFrame()
curr_team_data['Team'] = curr_home_data.index

curr_team_data = curr_team_data.merge(curr_home_data['HAS'], left_on='Team', right_index=True)
curr_team_data = curr_team_data.merge(curr_home_data['HAC'], left_on='Team', right_index=True)

curr_team_data = curr_team_data.merge(curr_away_data['AAS'], left_on='Team', right_index=True)
curr_team_data = curr_team_data.merge(curr_away_data['AAC'], left_on='Team', right_index=True)

curr_team_data.set_index("Team", inplace=True)
curr_season_table= pd.DataFrame(index=curr_team_data.index)
curr_team_data = team_data.merge(curr_team_data, on=['HAS', 'HAC', 'AAS', 'AAC'], left_index=True,
                                      right_index=True, how='outer')
curr_team_data = pd.DataFrame(scaler.fit_transform(curr_team_data), columns=curr_team_data.columns, index=curr_team_data.index)
curr_team_data

,HAS,HAC,AAS,AAC
Team,,,,
Arsenal,0.680851,0.214286,0.730769,0.705882
Aston Villa,0.526316,0.368421,0.631579,0.894737
Bournemouth,0.425532,0.535714,0.538462,1.000000
Brighton,0.191489,0.642857,0.153846,0.617647
Burnley,0.297872,0.785714,0.346154,0.735294
Cardiff,0.234043,1.000000,0.038462,0.588235
Chelsea,0.617021,0.071429,0.461538,0.470588
Crystal Palace,0.191489,0.464286,0.769231,0.558824
Everton,0.425532,0.392857,0.461538,0.411765


In [17]:
col_list2 = ['HomeTeam', 'AwayTeam']
data_prep = curr_season[col_list2]

data_prep = data_prep.merge(curr_team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
data_prep = data_prep.merge(curr_team_data['HAC'], left_on='HomeTeam', right_index=True, how='left') 
data_prep = data_prep.merge(curr_team_data['AAS'], left_on='AwayTeam', right_index=True, how='left') 
data_prep = data_prep.merge(curr_team_data['AAC'], left_on='AwayTeam', right_index=True, how='left') 

data_prep['HAtS'] = data_prep['HAS'] * data_prep['AAC']; # Home Attack Strength
data_prep['AAtS'] = data_prep['HAC'] * data_prep['AAS']; # Away Attack Strength

print(data_prep.shape)

data_prep = data_prep.merge(curr_season['Result'], left_index=True, right_index=True)

fixture_list = data_prep
data_prep

(148, 8)


,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,Liverpool,Norwich,0.957447,0.000000,0.210526,0.736842,0.705487,0.000000,-1
1,West Ham,Man City,0.468085,0.607143,1.000000,0.000000,0.000000,0.607143,1
2,Bournemouth,Sheffield United,0.425532,0.535714,0.421053,0.315789,0.134378,0.225564,0
3,Burnley,Southampton,0.297872,0.785714,0.230769,0.705882,0.210263,0.181319,-1
4,Crystal Palace,Everton,0.191489,0.464286,0.461538,0.411765,0.078849,0.214286,0
...,...,...,...,...,...,...,...,...,...
143,Leicester,Watford,0.297872,0.357143,0.538462,0.588235,0.175219,0.192308,-1
144,Man United,Tottenham,0.489362,0.535714,0.807692,0.352941,0.172716,0.432692,-1
145,Southampton,Norwich,0.361702,0.714286,0.210526,0.736842,0.266517,0.150376,-1
146,Wolves,West Ham,0.382979,0.392857,0.307692,0.500000,0.191489,0.120879,-1


In [18]:
rm_col_list = ['HomeTeam','AwayTeam']
col_list = data_prep.columns.values.tolist()
col_list = [x for x in col_list if x not in rm_col_list]

to_use = data_prep[col_list]
X = [x for x in col_list if x != 'Result']
Y = [x for x in col_list if x == 'Result']

X_data = to_use.drop('Result', axis=1)
Y_data = to_use['Result']

to_use

,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,0.957447,0.000000,0.210526,0.736842,0.705487,0.000000,-1
1,0.468085,0.607143,1.000000,0.000000,0.000000,0.607143,1
2,0.425532,0.535714,0.421053,0.315789,0.134378,0.225564,0
3,0.297872,0.785714,0.230769,0.705882,0.210263,0.181319,-1
4,0.191489,0.464286,0.461538,0.411765,0.078849,0.214286,0
...,...,...,...,...,...,...,...
143,0.297872,0.357143,0.538462,0.588235,0.175219,0.192308,-1
144,0.489362,0.535714,0.807692,0.352941,0.172716,0.432692,-1
145,0.361702,0.714286,0.210526,0.736842,0.266517,0.150376,-1
146,0.382979,0.392857,0.307692,0.500000,0.191489,0.120879,-1


In [19]:
predicted = model1.predict(X_data)
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,-1
1,1
2,1
3,-1
4,0


In [20]:
to_use = to_use.merge(predictedDF, left_index=True, right_index=True)
fixture_list = fixture_list.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result,Predicted
0,Liverpool,Norwich,0.957447,0.000000,0.210526,0.736842,0.705487,0.000000,-1,-1
1,West Ham,Man City,0.468085,0.607143,1.000000,0.000000,0.000000,0.607143,1,1
2,Bournemouth,Sheffield United,0.425532,0.535714,0.421053,0.315789,0.134378,0.225564,0,1
3,Burnley,Southampton,0.297872,0.785714,0.230769,0.705882,0.210263,0.181319,-1,-1
4,Crystal Palace,Everton,0.191489,0.464286,0.461538,0.411765,0.078849,0.214286,0,0
...,...,...,...,...,...,...,...,...,...,...
143,Leicester,Watford,0.297872,0.357143,0.538462,0.588235,0.175219,0.192308,-1,1
144,Man United,Tottenham,0.489362,0.535714,0.807692,0.352941,0.172716,0.432692,-1,1
145,Southampton,Norwich,0.361702,0.714286,0.210526,0.736842,0.266517,0.150376,-1,-1
146,Wolves,West Ham,0.382979,0.392857,0.307692,0.500000,0.191489,0.120879,-1,-1


In [21]:
curr_season_table['Actual Points'] = 0 
curr_season_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == -1:
        curr_season_table.loc[ht, which] += 3
    elif x == 0:
        curr_season_table.loc[ht, which] += 1
        curr_season_table.loc[at, which] += 1
    elif x == 1:
        curr_season_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Result, x.HomeTeam, x.AwayTeam, 'Actual Points'), axis=1)
fixture_list.apply(lambda x: make_actual_table(x.Predicted, x.HomeTeam, x.AwayTeam, 'Predicted Points'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
143    None
144    None
145    None
146    None
147    None
Length: 148, dtype: object

In [22]:
actual_index = curr_season_table.sort_values(by='Actual Points', ascending =0).index
predicted_index = curr_season_table.sort_values(by='Predicted Points', ascending =0).index
curr_season_table.sort_values(by='Actual Points', ascending =0)

,Actual Points,Predicted Points
Team,,
Liverpool,43,37
Leicester,35,21
Man City,32,42
Chelsea,29,32
Wolves,23,23
Crystal Palace,21,12
Man United,21,21
Tottenham,20,36
Sheffield United,19,30


In [23]:
acc(actual_index,predicted_index)
curr_season_table.sort_values(by='Predicted Points', ascending =0)

Accuracy of Predicted Table to Accurate Table 5.0%


,Actual Points,Predicted Points
Team,,
Man City,32,42
Liverpool,43,37
Tottenham,20,36
Chelsea,29,32
Arsenal,19,31
Sheffield United,19,30
Everton,14,25
Wolves,23,23
Leicester,35,21


In [24]:
curr_team_data = curr_team_data.rename(index={
    'Sheffield United':'Sheffield Utd',
    'Man United': 'Man Utd',
    'Tottenham':'Spurs'
})
print(curr_team_data.index.unique())
curr_team_data

Index(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton', 'Burnley',
       'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Huddersfield', 'Leicester', 'Liverpool', 'Man City', 'Man Utd',
       'Newcastle', 'Norwich', 'Sheffield Utd', 'Southampton', 'Spurs',
       'Watford', 'West Ham', 'Wolves'],
      dtype='object', name='Team')


,HAS,HAC,AAS,AAC
Team,,,,
Arsenal,0.680851,0.214286,0.730769,0.705882
Aston Villa,0.526316,0.368421,0.631579,0.894737
Bournemouth,0.425532,0.535714,0.538462,1.000000
Brighton,0.191489,0.642857,0.153846,0.617647
Burnley,0.297872,0.785714,0.346154,0.735294
Cardiff,0.234043,1.000000,0.038462,0.588235
Chelsea,0.617021,0.071429,0.461538,0.470588
Crystal Palace,0.191489,0.464286,0.769231,0.558824
Everton,0.425532,0.392857,0.461538,0.411765


In [25]:
curr_fixtures = pd.read_csv(loc + "epl-2019.csv", usecols=['Home Team', 'Away Team'])
curr_fixtures.columns = ['HomeTeam', 'AwayTeam']

curr_fixtures = curr_fixtures.merge(curr_team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
curr_fixtures = curr_fixtures.merge(curr_team_data['HAC'], left_on='HomeTeam', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['AAS'], left_on='AwayTeam', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['AAC'], left_on='AwayTeam', right_index=True, how='left') 

curr_fixtures['HAtS'] = curr_fixtures['HAS'] * curr_fixtures['AAC']; # Home Attack Strength
curr_fixtures['AAtS'] = curr_fixtures['HAC'] * curr_fixtures['AAS']; # Away Attack Strength

print(curr_fixtures.shape)

fixture_list = curr_fixtures
curr_fixtures

(380, 8)


,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS
0,Liverpool,Norwich,0.957447,0.000000,0.210526,0.736842,0.705487,0.000000
1,West Ham,Man City,0.468085,0.607143,1.000000,0.000000,0.000000,0.607143
2,Bournemouth,Sheffield Utd,0.425532,0.535714,0.421053,0.315789,0.134378,0.225564
3,Burnley,Southampton,0.297872,0.785714,0.230769,0.705882,0.210263,0.181319
4,Crystal Palace,Everton,0.191489,0.464286,0.461538,0.411765,0.078849,0.214286
...,...,...,...,...,...,...,...,...
375,Leicester,Man Utd,0.297872,0.357143,0.769231,0.529412,0.157697,0.274725
376,Man City,Norwich,1.000000,0.071429,0.210526,0.736842,0.736842,0.015038
377,Newcastle,Liverpool,0.297872,0.535714,0.846154,0.029412,0.008761,0.453297
378,Southampton,Sheffield Utd,0.361702,0.714286,0.421053,0.315789,0.114222,0.300752


In [26]:
col_list = [x for x in col_list if x not in ['Result']]
curr_fixtures = curr_fixtures[col_list]
curr_fixtures

,HAS,HAC,AAS,AAC,HAtS,AAtS
0,0.957447,0.000000,0.210526,0.736842,0.705487,0.000000
1,0.468085,0.607143,1.000000,0.000000,0.000000,0.607143
2,0.425532,0.535714,0.421053,0.315789,0.134378,0.225564
3,0.297872,0.785714,0.230769,0.705882,0.210263,0.181319
4,0.191489,0.464286,0.461538,0.411765,0.078849,0.214286
...,...,...,...,...,...,...
375,0.297872,0.357143,0.769231,0.529412,0.157697,0.274725
376,1.000000,0.071429,0.210526,0.736842,0.736842,0.015038
377,0.297872,0.535714,0.846154,0.029412,0.008761,0.453297
378,0.361702,0.714286,0.421053,0.315789,0.114222,0.300752


In [27]:
predicted = model1.predict(curr_fixtures)
predicted
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,-1
1,1
2,1
3,-1
4,0


In [28]:
fixture_list = fixture_list.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Predicted
0,Liverpool,Norwich,0.957447,0.000000,0.210526,0.736842,0.705487,0.000000,-1
1,West Ham,Man City,0.468085,0.607143,1.000000,0.000000,0.000000,0.607143,1
2,Bournemouth,Sheffield Utd,0.425532,0.535714,0.421053,0.315789,0.134378,0.225564,1
3,Burnley,Southampton,0.297872,0.785714,0.230769,0.705882,0.210263,0.181319,-1
4,Crystal Palace,Everton,0.191489,0.464286,0.461538,0.411765,0.078849,0.214286,0
...,...,...,...,...,...,...,...,...,...
375,Leicester,Man Utd,0.297872,0.357143,0.769231,0.529412,0.157697,0.274725,1
376,Man City,Norwich,1.000000,0.071429,0.210526,0.736842,0.736842,0.015038,-1
377,Newcastle,Liverpool,0.297872,0.535714,0.846154,0.029412,0.008761,0.453297,1
378,Southampton,Sheffield Utd,0.361702,0.714286,0.421053,0.315789,0.114222,0.300752,-1


In [29]:
final_predicted_table= pd.DataFrame(index=fixture_list.HomeTeam.unique())
final_predicted_table['Predicted Points'] = 0

In [30]:
final_predicted_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == -1:
        final_predicted_table.loc[ht, which] += 3
    elif x == 0:
        final_predicted_table.loc[ht, which] += 1
        final_predicted_table.loc[at, which] += 1
    elif x == 1:
        final_predicted_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Predicted, x.HomeTeam, x.AwayTeam, 'Predicted Points'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

## Predcited 19/20 Season Table
Based on our model, this is how the table will look like at the end of the current season
The stats we used for some of the teams is from the last season so our model might not perform upto expectations.
We could also simulate the rest of the season using the stats by looking at the matches played till Matchday 15, but for now, this should do.

In [31]:
final_predicted_table.sort_values('Predicted Points', ascending=0)

,Predicted Points
Man City,108
Liverpool,95
Spurs,88
Arsenal,72
Sheffield Utd,71
Chelsea,71
Aston Villa,62
Everton,62
Man Utd,60
West Ham,55
